In [ ]:
!pip install -U scikit-learn transformers accelerate opencv-python pillow qwen-vl-utils av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 73.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 87.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/88

# DAiSEE Student Expression Recognition with Qwen2.5-VL-7B

This notebook performs inference on the DAiSEE dataset Test set using the Qwen2.5-VL-7B-Instruct model to recognize spontaneous student expressions.

**Dataset**: DAiSEE (Dataset for Affective States in E-learning Environments)
- **Labels**: Boredom, Engagement, Confusion, Frustration (levels 0-3)
- **Test Set Path**: `/kaggle/input/daisee/DAiSEE/DataSet/Test/`
- **Labels Path**: `/kaggle/input/daisee/DAiSEE/Labels/TestLabels.csv`

**Model**: Qwen/Qwen2.5-VL-7B-Instruct

**Temporal Sampling**: 1 FPS (1 frame per second) - handled by the model's processor

**Evaluation Metrics**: F1 Score, Accuracy, Precision, Recall

In [1]:
import os

In [2]:
import cv2

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
from pathlib import Path

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
from sklearn.metrics import precision_recall_fscore_support

In [8]:
from sklearn.metrics import confusion_matrix

In [9]:
import torch

In [12]:
from transformers import AutoProcessor, AutoModelForVision2Seq

In [14]:
import gc

In [ ]:
# Configuration
TEST_DATA_PATH = "/kaggle/input/daisee/DAiSEE/DataSet/Test"
LABELS_PATH = "/kaggle/input/daisee/DAiSEE/Labels/TestLabels.csv"
TEMPORAL_SAMPLING_RATE = 1  # 1 FPS
MODEL_NAME = "Qwen/Qwen2.5-VL-7B-Instruct"

# DAiSEE label categories and levels
# Note: Frustration has a trailing space to match the CSV column name
LABEL_CATEGORIES = ["Boredom", "Engagement", "Confusion", "Frustration "]
LABEL_LEVELS = [0, 1, 2, 3]

## Load Labels and Initialize Model

In [16]:
# Load test labels
labels_df = pd.read_csv(LABELS_PATH)
print(f"Total test samples: {len(labels_df)}")
print(f"\nColumn names in CSV:")
print(labels_df.columns.tolist())
print(f"\nFirst few rows:")
print(labels_df.head())
print(f"\nLabel distribution:")
for category in LABEL_CATEGORIES:
    if category in labels_df.columns:
        print(f"\n{category}:")
        print(labels_df[category].value_counts().sort_index())
    else:
        print(f"\n{category}: Column not found in CSV!")

Total test samples: 1784

Column names in CSV:
['ClipID', 'Boredom', 'Engagement', 'Confusion', 'Frustration ']

First few rows:
           ClipID  Boredom  Engagement  Confusion  Frustration 
0  5000441001.avi        1           2          0             0
1  5000441002.avi        0           2          0             0
2  5000441003.avi        1           2          0             0
3  5000441005.avi        2           2          0             0
4  5000441006.avi        2           2          1             2

Label distribution:

Boredom:
Boredom
0    823
1    584
2    338
3     39
Name: count, dtype: int64

Engagement:
Engagement
0      4
1     84
2    882
3    814
Name: count, dtype: int64

Confusion:
Confusion
0    1200
1     427
2     136
3      21
Name: count, dtype: int64

Frustration :
Frustration 
0    1388
1     316
2      57
3      23
Name: count, dtype: int64


In [17]:
# Load model and processor
print("Loading Qwen2.5-VL-7B-Instruct model...")
processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)
print(f"Model loaded successfully on {'GPU' if torch.cuda.is_available() else 'CPU'}")

Loading Qwen2.5-VL-7B-Instruct model...


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Model loaded successfully on GPU


## Define Prompt Templates for DAiSEE

In [ ]:
def create_combined_prompt():
    """
    Create a single combined prompt for all DAiSEE categories to minimize inference calls.
    Returns a prompt that asks for all four labels at once.
    """
    
    prompt = """Context:
The student in the video is watching a teaching video. Rate the student's expression across four categories, each on a scale from 0 to 3:

**Boredom:**
• Level 0: Not bored at all - the student appears fully alert and interested
• Level 1: Slightly bored - the student shows minor signs of disinterest
• Level 2: Moderately bored - the student appears clearly disengaged or restless
• Level 3: Extremely bored - the student shows strong signs of disengagement, such as looking away frequently or appearing very uninterested

**Engagement:**
• Level 0: Not engaged - the student appears completely disinterested and unfocused
• Level 1: Slightly engaged - the student shows minimal attention to the content
• Level 2: Moderately engaged - the student appears reasonably attentive and focused
• Level 3: Highly engaged - the student shows strong focus and active interest in the content

**Confusion:**
• Level 0: Not confused - the student appears to understand clearly
• Level 1: Slightly confused - the student shows minor signs of puzzlement
• Level 2: Moderately confused - the student appears clearly puzzled, may be frowning or showing uncertainty
• Level 3: Extremely confused - the student shows strong signs of confusion and difficulty understanding

**Frustration:**
• Level 0: Not frustrated - the student appears calm and composed
• Level 1: Slightly frustrated - the student shows minor signs of irritation or impatience
• Level 2: Moderately frustrated - the student appears clearly frustrated or annoyed
• Level 3: Extremely frustrated - the student shows strong signs of frustration, such as distress or agitation

Provide your answer in the following exact format (one number per line):
Boredom: [0-3]
Engagement: [0-3]
Confusion: [0-3]
Frustration: [0-3]"""
    
    return prompt

# Test the prompt generation
print("Combined Prompt:")
print("="*60)
print(create_combined_prompt())


Prompt for Boredom:
Context:
The student in the image is watching a teaching video. Rate the student's level of boredom on a scale from 0 to 3:
• Level 0: Not bored at all - the student appears fully alert and interested
• Level 1: Slightly bored - the student shows minor signs of disinterest
• Level 2: Moderately bored - the student appears clearly disengaged or restless
• Level 3: Extremely bored - the student shows strong signs of disengagement, such as looking away frequently or appearing very uninterested

Provide only the level number (0, 1, 2, or 3) as your answer, without any additional words.

Prompt for Engagement:
Context:
The student in the image is watching a teaching video. Rate the student's level of engagement on a scale from 0 to 3:
• Level 0: Not engaged - the student appears completely disinterested and unfocused
• Level 1: Slightly engaged - the student shows minimal attention to the content
• Level 2: Moderately engaged - the student appears reasonably attentive a

## Video Processing Functions

In [18]:
def find_video_path(clip_id, test_data_path):
    """
    Find the full path to a video file given its ClipID.
    
    Structure: Test -> Subject -> Video Folder -> Video File
    Example: Test/500044/5000441001/5000441001.avi
    """
    test_path = Path(test_data_path)
    
    # Extract subject ID from clip_id (first 6 digits)
    # For example: 5000441001 -> 500044
    clip_name = clip_id.replace('.avi', '')
    subject_id = clip_name[:6]
    
    # Construct the full path
    video_path = test_path / subject_id / clip_name / clip_id
    
    return video_path

## Inference Function

In [ ]:
def predict_all_categories(video_path, processor, model, fps=1):
    """
    Predict levels (0-3) for all categories using a single inference call.
    
    Args:
        video_path: Path to the video file
        processor: The AutoProcessor
        model: The AutoModelForVision2Seq
        fps: Frames per second for temporal sampling
    
    Returns:
        Dictionary with predicted levels for all categories and the raw response
    """
    prompt_text = create_combined_prompt()
    
    try:
        # Create message for the model with video input
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": str(video_path)},
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]
        
        # Process inputs with fps parameter
        inputs = processor.apply_chat_template(
            messages,
            fps=fps,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device)
        
        # Generate response
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=100)
        
        # Decode the response
        response = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        
        # Parse the levels from response
        predictions = {}
        for category in LABEL_CATEGORIES:
            try:
                # Look for pattern like "Boredom: 2" or "Boredom:2"
                import re
                pattern = rf"{category.strip()}\s*:\s*(\d)"
                match = re.search(pattern, response, re.IGNORECASE)
                if match:
                    level = int(match.group(1))
                    if level not in [0, 1, 2, 3]:
                        level = 0
                else:
                    level = 0  # Default if not found
            except:
                level = 0
            
            predictions[category] = level
        
        return predictions, response
        
    except Exception as e:
        print(f"Error processing video: {e}")
        # Return default values
        return {cat: 0 for cat in LABEL_CATEGORIES}, str(e)        

## Run Inference on Test Set

In [ ]:
def run_inference_on_test_set(labels_df, test_data_path, processor, model, temporal_fps=1):
    """
    Run inference on all videos in the test set.
    
    Args:
        labels_df: DataFrame with ground truth labels
        test_data_path: Path to test dataset
        processor: The AutoProcessor
        model: The AutoModelForVision2Seq
        temporal_fps: Frames per second for temporal sampling
    
    Returns:
        DataFrame with predictions and ground truth
    """
    results = []
    total_videos = len(labels_df)
    
    for idx, row in labels_df.iterrows():
        clip_id = row['ClipID']
        
        # Print progress every 10 videos
        if (idx + 1) % 10 == 0 or idx == 0:
            print(f"\n[{idx+1}/{total_videos}] Progress: {(idx + 1) / total_videos * 100:.1f}%")
        
        # Find video path
        video_path = find_video_path(clip_id, test_data_path)
        
        if not video_path.exists():
            print(f"  Video not found: {clip_id}")
            continue
        
        # Predict all categories in a single inference call
        predictions_dict, response = predict_all_categories(video_path, processor, model, fps=temporal_fps)
        
        # Prepare result row
        result = {"ClipID": clip_id}
        
        for category in LABEL_CATEGORIES:
            pred_level = predictions_dict[category]
            true_level = row[category]
            result[f"{category}_pred"] = pred_level
            result[f"{category}_true"] = true_level
        
        results.append(result)
        
        # Clear memory
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    results_df = pd.DataFrame(results)
    return results_df

In [22]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 44.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
# Run inference on the entire test set
print("Starting inference on DAiSEE Test set...")
print(f"Temporal sampling rate: {TEMPORAL_SAMPLING_RATE} FPS")
print(f"Total videos to process: {len(labels_df)}")
print("-" * 60)

results_df = run_inference_on_test_set(
    labels_df=labels_df,
    test_data_path=TEST_DATA_PATH,
    processor=processor,
    model=model,
    temporal_fps=TEMPORAL_SAMPLING_RATE
)

print("\n" + "=" * 60)
print(f"Inference completed on {len(results_df)} videos")
print("=" * 60)
print("\nFirst few results:")
print(results_df.head())

Starting inference on DAiSEE Test set...
Temporal sampling rate: 1 FPS

[1/1784] Processing: 5000441001.avi
  Predicting Boredom...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 1, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Engagement...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 2, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Confusion...
Error processing video: You chose backend=pyav for loadin

Exception ignored in: <function _xla_gc_callback at 0x787b80627880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Progress: 2.6%

[48/1784] Processing: 5000441061.avi
  Predicting Boredom...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 1, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Engagement...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 3, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Confusion...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in yo

Exception ignored in: <function _xla_gc_callback at 0x787b80627880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Progress: 2.9%

[52/1784] Processing: 5000441066.avi
  Predicting Boredom...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 0, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Engagement...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 2, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Confusion...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in yo

Exception ignored in: <function _xla_gc_callback at 0x787b80627880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Progress: 58.2%

[1039/1784] Processing: 5100451056.avi
  Predicting Boredom...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 1, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Engagement...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
    Predicted: 0, Ground Truth: 3, Response: You chose backend=pyav for loading the video but the required library is not found in your environment Make sure to install pyav before loading the video.
  Predicting Confusion...
Error processing video: You chose backend=pyav for loading the video but the required library is not found in

## Evaluation Metrics

In [ ]:
def calculate_metrics(y_true, y_pred, category_name):
    """
    Calculate evaluation metrics: Accuracy, Precision, Recall, F1 Score.
    
    Args:
        y_true: Ground truth labels
        y_pred: Predicted labels
        category_name: Name of the category for logging
    
    Returns:
        Dictionary with metrics
    """
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    # Precision, Recall, F1 with macro averaging (treats all classes equally)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    
    # Also calculate weighted metrics (accounts for class imbalance)
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0
    )
    
    # Per-class metrics
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0, labels=[0, 1, 2, 3]
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
    
    metrics = {
        'category': category_name,
        'accuracy': accuracy,
        'precision_macro': precision,
        'recall_macro': recall,
        'f1_macro': f1,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'confusion_matrix': cm,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class,
        'support_per_class': support_per_class
    }
    
    return metrics


def print_metrics(metrics):
    """Print metrics in a formatted way."""
    print(f"\n{'='*70}")
    print(f"Metrics for {metrics['category']}")
    print('='*70)
    print(f"Accuracy:           {metrics['accuracy']:.4f}")
    print(f"\nMacro-averaged metrics:")
    print(f"  Precision:        {metrics['precision_macro']:.4f}")
    print(f"  Recall:           {metrics['recall_macro']:.4f}")
    print(f"  F1 Score:         {metrics['f1_macro']:.4f}")
    print(f"\nWeighted-averaged metrics:")
    print(f"  Precision:        {metrics['precision_weighted']:.4f}")
    print(f"  Recall:           {metrics['recall_weighted']:.4f}")
    print(f"  F1 Score:         {metrics['f1_weighted']:.4f}")
    
    print(f"\nPer-class metrics:")
    print(f"{'Level':<10} {'Precision':<12} {'Recall':<12} {'F1':<12} {'Support':<10}")
    print('-'*60)
    for level in range(4):
        print(f"{level:<10} {metrics['precision_per_class'][level]:<12.4f} "
              f"{metrics['recall_per_class'][level]:<12.4f} "
              f"{metrics['f1_per_class'][level]:<12.4f} "
              f"{int(metrics['support_per_class'][level]):<10}")
    
    print(f"\nConfusion Matrix:")
    print(f"{'':>10} {'Pred 0':<10} {'Pred 1':<10} {'Pred 2':<10} {'Pred 3':<10}")
    for i, row in enumerate(metrics['confusion_matrix']):
        print(f"True {i}:   {row[0]:<10} {row[1]:<10} {row[2]:<10} {row[3]:<10}")
    print('='*70)

In [ ]:
# Calculate metrics for each category
all_metrics = {}

for category in LABEL_CATEGORIES:
    y_true = results_df[f"{category}_true"].values
    y_pred = results_df[f"{category}_pred"].values
    
    metrics = calculate_metrics(y_true, y_pred, category)
    all_metrics[category] = metrics
    
    # Print metrics
    print_metrics(metrics)

## Summary Statistics and Overall Performance

In [ ]:
# Create summary table
summary_data = []
for category in LABEL_CATEGORIES:
    metrics = all_metrics[category]
    summary_data.append({
        'Category': category,
        'Accuracy': f"{metrics['accuracy']:.4f}",
        'Precision (Macro)': f"{metrics['precision_macro']:.4f}",
        'Recall (Macro)': f"{metrics['recall_macro']:.4f}",
        'F1 Score (Macro)': f"{metrics['f1_macro']:.4f}",
        'F1 Score (Weighted)': f"{metrics['f1_weighted']:.4f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("OVERALL PERFORMANCE SUMMARY")
print("="*100)
print(summary_df.to_string(index=False))
print("="*100)

# Calculate average metrics across all categories
avg_accuracy = np.mean([all_metrics[cat]['accuracy'] for cat in LABEL_CATEGORIES])
avg_precision = np.mean([all_metrics[cat]['precision_macro'] for cat in LABEL_CATEGORIES])
avg_recall = np.mean([all_metrics[cat]['recall_macro'] for cat in LABEL_CATEGORIES])
avg_f1 = np.mean([all_metrics[cat]['f1_macro'] for cat in LABEL_CATEGORIES])

print(f"\nAverage across all categories:")
print(f"  Accuracy:  {avg_accuracy:.4f}")
print(f"  Precision: {avg_precision:.4f}")
print(f"  Recall:    {avg_recall:.4f}")
print(f"  F1 Score:  {avg_f1:.4f}")

## Save Results

In [ ]:
# Save predictions to CSV
output_path = "daisee_test_predictions.csv"
results_df.to_csv(output_path, index=False)
print(f"\nPredictions saved to: {output_path}")

# Save summary metrics to CSV
summary_output_path = "daisee_test_summary_metrics.csv"
summary_df.to_csv(summary_output_path, index=False)
print(f"Summary metrics saved to: {summary_output_path}")

print("\nAll results saved locally.")